In [22]:
import requests
import json
import pandas as pd
from pprint import pprint
from time import sleep
import os

from spotify_credentials import client_id, client_secret

In [23]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_key = r['access_token']



In [27]:
songdf = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\master_data_final.csv")
songdf["explicit"] = False

songdf = songdf.iloc[:,[6,2,3,4,5,7,8,9,10,11,12,13,14,15]]

iter_df = songdf.iloc[:,[0,13]]

In [28]:

spotify_endpoint = "https://api.spotify.com/v1/tracks/"
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_key)}

for index, row in iter_df.iterrows():

    song_id = str(iter_df.iat[index,0])

    try:
        spotify_response = requests.get((spotify_endpoint + song_id), headers = spotify_headers).json()
        songdf.iat[index,13] = spotify_response["explicit"]
        sleep(0.25)
        
    except:
        if spotify_response == {'error': {'status': 429}}:
            print(f"{spotify_response} on song ID = {song_id}")
        else:
            print(f"Failure on ID: {song_id}")
        pass

        




path = r"C:\Users\kevin\bootcamp\Project_1\resources"
songdf.to_csv(os.path.join(path,f"master_data_final_reference.csv"),index=False) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

songdf.head()

Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan
Failure on ID: nan


,id,title,artist,count,year,duration,loudness,tempo,time_signature,key,danceability,energy,valence,explicit
0,17phhZDn6oGtzMe56NuWvj,Lose Control,Teddy Swims,43,2024,210.68884,-4.409,159.920,3.0,9.0,0.561,0.604,0.242,False
1,0mflMxspEfB0VbI1kyLiAv,Stick Season,Noah Kahan,40,2024,182.34671,-6.935,117.896,4.0,9.0,0.664,0.500,0.801,False
2,3xkHsmpQCBMytMJNiDf3Ii,Beautiful Things,Benson Boone,39,2024,180.30449,-5.692,105.029,3.0,10.0,0.472,0.471,0.219,False
3,6D8y7Bck8h11byRY88Pt2z,Houdini,Dua Lipa,37,2024,185.91795,-4.876,116.985,4.0,9.0,0.744,0.789,0.866,False
4,4KULAymBBJcPRpk1yO4dOG,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,35,2024,227.19592,-7.746,77.639,4.0,0.0,0.429,0.453,0.155,False
